# Multi-Agent System Demo

This notebook demonstrates how to set up and use a multi-agent system using Google's Agent Development Kit (ADK). The system consists of specialized sub-agents (Billing and Support) coordinated by a main agent.

## Import Required Libraries

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.genai import types
import google.generativeai as genai
from google.adk.models.lite_llm import LiteLlm
import os
import logging
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.colors import LinearSegmentedColormap

## Configure Logging

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger("<<!!!!!!!>>")

## Configure API Keys

**Note:** In a production environment, you should use environment variables or a secure configuration method rather than hardcoding API keys.

In [ ]:
genai.configure(api_key="")
os.environ['OPENAI_API_KEY'] = ''
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
MODEL_GPT_4O = "gpt-3.5-turbo-0125"

## Define Specialized Sub-Agents

We'll create two specialized agents:
1. Billing Agent - Handles billing and payment-related inquiries
2. Support Agent - Provides technical support and troubleshooting assistance

In [ ]:
# Define specialized sub-agents
billing_agent = LlmAgent(
    name="Billing",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="You handle billing and payment-related inquiries.",
    description="Handles billing inquiries."
)

support_agent = LlmAgent(
    name="Support",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="You provide technical support and troubleshooting assistance.",
    description="Handles technical support requests."
)

## Define the Coordinator Agent

The coordinator agent routes user requests to the appropriate specialized agent.

In [ ]:
# Define the coordinator agent
coordinator = LlmAgent(
    name="HelpDeskCoordinator",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="Route user requests: Use Billing agent for payment issues, Support agent for technical problems.",
    description="Main help desk router.",
    sub_agents=[billing_agent, support_agent]
)

# For ADK compatibility, the root agent must be named `root_agent`
root_agent = coordinator

## Set Up the Runner

The Runner is responsible for managing the agent execution, sessions, and artifacts.

In [ ]:
runner = Runner(
        app_name="test_agent",
        agent=root_agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),)

## Run a Simulation with a User Query

Let's test our multi-agent system with a sample user query. You can uncomment the query you want to test.

In [ ]:
# Simulate a user query
user_query = "I can't log in gmail, help to give advice."
# user_query = "my billing is not working, help to give advice."
user_id = "test_user"
session_id = "test_session"

# Create a session
session = runner.session_service.create_session(
    app_name="test_agent",
    user_id=user_id,
    state={},
    session_id=session_id,
)

In [ ]:
# Create a content object with the user query
content = types.Content(
    role="user", 
    parts=[types.Part.from_text(text=user_query)]
)

# Run the agent with the correct parameters
events = list(runner.run(
    user_id=user_id, 
    session_id=session.id, 
    new_message=content
))

In [ ]:
# Process the events to get the response
response = ""
if events and events[-1].content and events[-1].content.parts:
    for event in events:
        logger.info(f"Event: {event.author}, Actions: {event.actions}")
        response = "\n".join([p.text for p in events[-1].content.parts if p.text])

print(response)

## Try Different Queries

You can experiment with different user queries to see how the multi-agent system routes and handles them. For example:
- Technical support queries should be routed to the Support agent
- Billing and payment queries should be routed to the Billing agent

In [ ]:
# Function to test different queries
def test_query(query):
    # Create a new session for each test
    test_session_id = f"test_session_{hash(query)}"
    session = runner.session_service.create_session(
        app_name="test_agent",
        user_id=user_id,
        state={},
        session_id=test_session_id,
    )
    
    # Create content and run the agent
    content = types.Content(role="user", parts=[types.Part.from_text(text=query)])
    events = list(runner.run(user_id=user_id, session_id=session.id, new_message=content))
    
    # Process and return the response and events
    if events and events[-1].content and events[-1].content.parts:
        for event in events:
            logger.info(f"Event: {event.author}, Actions: {event.actions}")
        response = "\n".join([p.text for p in events[-1].content.parts if p.text])
        return response, events
    return "No response", []

# Helper function to extract agent invoke path from events
def extract_agent_path(events):
    agent_path = []
    for event in events:
        if hasattr(event, 'author') and event.author:
            agent_path.append(event.author)
    return agent_path

In [ ]:
# Test with a technical support query
tech_query = "I can't access my email account"
print(f"Query: {tech_query}")
tech_response, tech_events = test_query(tech_query)
print(f"Response: {tech_response}")
tech_path = extract_agent_path(tech_events)
print(f"Agent Path: {tech_path}")

In [ ]:
# Test with a billing query
billing_query = "I was charged twice for my subscription"
print(f"Query: {billing_query}")
billing_response, billing_events = test_query(billing_query)
print(f"Response: {billing_response}")
billing_path = extract_agent_path(billing_events)
print(f"Agent Path: {billing_path}")

## Visualize Agent Invoke Path

Let's create a chart to visualize the agent invoke path for different queries.

In [ ]:
def visualize_agent_path(query_paths):
    """
    Visualize the agent invoke path for different queries using a directed graph.
    
    Args:
        query_paths: Dictionary mapping query descriptions to agent paths
    """
    plt.figure(figsize=(12, 8))
    
    # Create a directed graph
    G = nx.DiGraph()
    
    # Define node positions manually for better layout
    pos = {
        "User": (0, 0),
        "HelpDeskCoordinator": (1, 0),
        "Support": (2, 0.5),
        "Billing": (2, -0.5)
    }
    
    # Define colors for different query types
    colors = {
        "Technical Support Query": "blue",
        "Billing Query": "green"
    }
    
    # Add nodes
    G.add_node("User", desc="User")
    G.add_node("HelpDeskCoordinator", desc="Coordinator")
    G.add_node("Support", desc="Support Agent")
    G.add_node("Billing", desc="Billing Agent")
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=3000, node_color="lightgray")
    
    # Draw node labels
    nx.draw_networkx_labels(G, pos, labels={n: G.nodes[n]['desc'] for n in G.nodes()})
    
    # Add edges for each query path
    edge_colors = []
    edge_labels = {}
    edges = []
    
    for i, (query_desc, path) in enumerate(query_paths.items()):
        # Insert "User" at the beginning of the path
        full_path = ["User"] + path
        
        # Add edges for this path
        for j in range(len(full_path) - 1):
            source = full_path[j]
            target = full_path[j+1]
            edge = (source, target)
            edges.append(edge)
            edge_colors.append(colors[query_desc])
            
            # Add or update edge label
            if edge in edge_labels:
                edge_labels[edge] += f", {query_desc}"
            else:
                edge_labels[edge] = f"{query_desc}"
    
    # Add edges to the graph
    G.add_edges_from(edges)
    
    # Draw edges with colors
    for i, (u, v) in enumerate(edges):
        nx.draw_networkx_edges(G, pos, edgelist=[(u, v)], width=2, 
                              edge_color=edge_colors[i], arrows=True, 
                              arrowstyle='->', arrowsize=20)
    
    # Draw edge labels
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)
    
    # Add legend
    legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=query_type)
                      for query_type, color in colors.items()]
    plt.legend(handles=legend_elements, loc='upper right')
    
    plt.title("Agent Invoke Path for Different Queries", fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Create a dictionary of query paths
query_paths = {
    "Technical Support Query": tech_path,
    "Billing Query": billing_path
}

# Visualize the agent invoke paths
visualize_agent_path(query_paths)

## Alternative Visualization: Sequence Diagram

Let's also create a sequence diagram to show the flow of messages between agents.

In [ ]:
def plot_sequence_diagram(query_paths):
    """
    Create a sequence diagram showing the flow of messages between agents.
    
    Args:
        query_paths: Dictionary mapping query descriptions to agent paths
    """
    plt.figure(figsize=(12, 8))
    
    # Define all agents
    agents = ["User", "HelpDeskCoordinator", "Support", "Billing"]
    
    # Define x-positions for each agent
    x_positions = {agent: i for i, agent in enumerate(agents)}
    
    # Define colors for different query types
    colors = {
        "Technical Support Query": "blue",
        "Billing Query": "green"
    }
    
    # Draw vertical lines for each agent
    for agent, x in x_positions.items():
        plt.axvline(x=x, color='gray', linestyle='--', alpha=0.5)
        plt.text(x, -0.5, agent, ha='center', fontsize=12)
    
    # Plot each query path
    y_offset = 0
    for query_desc, path in query_paths.items():
        # Insert "User" at the beginning of the path
        full_path = ["User"] + path
        
        # Plot arrows between agents
        for i in range(len(full_path) - 1):
            source = full_path[i]
            target = full_path[i+1]
            
            # Get x-positions
            x_source = x_positions[source]
            x_target = x_positions[target]
            
            # Calculate y-position
            y = i + y_offset
            
            # Draw arrow
            plt.arrow(x_source, y, x_target - x_source, 0, 
                     head_width=0.1, head_length=0.1, fc=colors[query_desc], ec=colors[query_desc],
                     length_includes_head=True)
            
            # Add label
            plt.text((x_source + x_target) / 2, y + 0.15, query_desc, ha='center', fontsize=8)
        
        y_offset += len(full_path) + 1
    
    # Add legend
    legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=query_type)
                      for query_type, color in colors.items()]
    plt.legend(handles=legend_elements, loc='upper right')
    
    plt.title("Agent Sequence Diagram", fontsize=16)
    plt.ylim(-1, y_offset)
    plt.xlim(-0.5, len(agents) - 0.5)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the sequence diagram
plot_sequence_diagram(query_paths)